In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

# read in all files
train2015_05_28 = pd.read_csv("train2015_05_28_changed.csv")
train2015_06_28 = pd.read_csv("train2015_06_28_changed.csv")
train2016_05_28 = pd.read_csv("train2016_05_28.csv")
testX = pd.read_csv("testX.csv")

print(train2015_05_28.shape)
print(train2015_06_28.shape)
print(train2016_05_28.shape)
print(testX.shape)

# the unique ones in either 2015-05-28 or 2015-06-28
train2015_05_28_unique = pd.read_csv('train2015_05_28_unique.csv')
print("Unique ones are: ", train2015_05_28_unique.shape)

train2015_06_28_unique = pd.read_csv('train2015_06_28_unique.csv')
print("Unique ones are: ", train2015_06_28_unique.shape)

# read in changed id list
#changed_ids = []
#with open('changed_ids.txt', 'r') as f:
#    for line in f:
#        changed_ids.append(int(line.strip()))
#print("changed list is: ", len(changed_ids))

(51489, 47)
(51489, 47)
(931453, 47)
(929615, 23)
Unique ones are:  (3354, 47)
Unique ones are:  (3507, 47)


#### only 22 products has data. Products 'ind_ahor_fin_ult1' un 'ind_aval_fin_ult1' have not been purchased.
- so we need to drop these two columns
- in addition, the date information should also be removed!

In [3]:

droplist = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t', 'conyuemp', 'tipodom']
train2015_05_28.drop(droplist, 1, inplace=True)
train2015_06_28.drop(droplist, 1, inplace=True)
train2016_05_28.drop(droplist, 1, inplace=True)
print(train2015_05_28.shape)
print(train2015_06_28.shape)
print(train2016_05_28.shape)

train2015_05_28_unique.drop(droplist, 1, inplace=True)
train2015_06_28_unique.drop(droplist, 1, inplace=True)
print("Unique ones are: ", train2015_05_28_unique.shape)
print("Unique ones are: ", train2015_06_28_unique.shape)

(51489, 40)
(51489, 40)
(931453, 40)
Unique ones are:  (3354, 40)
Unique ones are:  (3507, 40)


#### It seems that antiguedad (antiquity or Seniority) have negative values -999999, so need to replace it with 0

In [4]:
train2015_05_28.loc[train2015_05_28.antiguedad < 0, 'antiguedad'] = 0
train2015_06_28.loc[train2015_06_28.antiguedad < 0, 'antiguedad'] = 0
train2016_05_28.loc[train2016_05_28.antiguedad < 0, 'antiguedad'] = 0
print(train2015_05_28.shape)
print(train2015_06_28.shape)
print(train2016_05_28.shape)

train2015_05_28_unique.loc[train2015_05_28_unique.antiguedad < 0, 'antiguedad'] = 0
train2015_06_28_unique.loc[train2015_06_28_unique.antiguedad < 0, 'antiguedad'] = 0
print("Unique ones are: ", train2015_05_28_unique.shape)
print("Unique ones are: ", train2015_06_28_unique.shape)

(51489, 40)
(51489, 40)
(931453, 40)
Unique ones are:  (3354, 40)
Unique ones are:  (3507, 40)


#### For padding
- when compare two different months, some columns values are missing as they are unique to individual months
- in order to do the comparison, the month with missing customers will have to pad everything in.

In [5]:
target_cols = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
               'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1',
               'ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
               'ind_nom_pens_ult1','ind_recibo_ult1']

# for padding
padding_for_55 = train2015_06_28_unique.copy()
padding_for_55[target_cols] = [0] * 22

#padding_for_56 = train2015_05_28_unique.copy()
#padding_for_56[target_cols] = [0] * 22


print("padding for 55 is: ", padding_for_55.shape)
#print(train2015_06_28_unique[target_cols].iloc[1:2,:])
#print(padding_for_55[target_cols].iloc[1:2,:])

# compose the changed list for 2015-05-28
#train55 = pd.concat([train2015_05_28_unique])
train55 = pd.concat([train2015_05_28, padding_for_55])
#train55 = pd.concat([train2015_05_28, train2015_05_28_unique, padding_for_55])
print("2015-05-28 only: ", train55.shape)

# compose the changed list for 2015-06-28
train56 = pd.concat([train2015_06_28, train2015_06_28_unique])
#train56 = pd.concat([train2015_06_28, train2015_06_28_unique, padding_for_56])
#train56 = pd.concat([padding_for_56])
print("2015-06-28 only: ", train56.shape)

# changed list for all 2015
trainX2015 = pd.concat([train2015_05_28, train2015_05_28_unique, train2015_06_28_unique])
print("the entire list for 2015: ", trainX2015.shape)

padding for 55 is:  (3507, 40)
2015-05-28 only:  (54996, 40)
2015-06-28 only:  (54996, 40)
the entire list for 2015:  (58350, 40)


In [ ]:
# now need to find out changed target in a single column list format

trainY = []
trainX = []

for idx, row in train56.iterrows():
    customer_id = row.ncodpers
    row55 = train55[train55.ncodpers == customer_id]        
    cur_target_list  = row[target_cols].values
    prev_target_list = row55[target_cols].values[0]
    row55.drop(['ncodpers'], 1, inplace=True)
    row55 = row55.values[0]
    #print(prev_target_list, " and ", cur_target_list)

    new_target_list =[max(abs(int(x1) - int(x2)), 0) for (x1, x2) in zip(cur_target_list, prev_target_list)]
    
    for ind, val in enumerate(new_target_list):
        if val != 0:
            trainX.append(row55)
            trainY.append(ind)
            
print(len(trainY))
print(len(trainX))

In [5]:
from collections import Counter
print(Counter(trainY))

trainX = np.array(trainX)
trainY = np.array(trainY)
print(trainY.shape)
print(trainX.shape)
print(trainX[0])

trainY.dump("trainY.dat")
trainX.dump("trainX.dat")

Counter({21: 16094, 0: 14624, 20: 11213, 16: 8840, 19: 8181, 2: 7414, 15: 2942, 9: 2192, 10: 1946, 7: 1278, 4: 874, 5: 789, 17: 598, 11: 522, 6: 465, 14: 318, 3: 161, 8: 98, 12: 84, 13: 42, 18: 23, 1: 15})
(78713,)
(78713, 41)
[  2.30000000e+01   0.00000000e+00   3.40000000e+01   1.00000000e+00
   1.00000000e+00   1.00000000e+00   2.80000000e+01   0.00000000e+00
   1.78865280e+05   0.00000000e+00   0.00000000e+00   1.00000000e+00
   1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.00000000e+00]


In [9]:
import pandas as pd
import numpy as np

# now need to convert testX file
testX = pd.read_csv("testX.csv")#, nrows=5)
print(testX.shape)
print(type(train2016_05_28))
print(train2016_05_28.shape)

#testX[target_cols] = [0]*22
test_ids = testX['ncodpers']
testX = train2016_05_28[train2016_05_28.ncodpers.isin(test_ids)]
#testX.drop(['ncodpers', 'fecha_dato', 'fecha_alta', 'ult_fec_cli_1t'], 1, inplace=True)

print(testX.shape)

test_ids = testX['ncodpers']
testX.drop(['ncodpers'], 1, inplace=True)
print("the test ids are: ", len(test_ids))

testX = np.array(testX)
print(testX.shape)

np.save("test_ids", test_ids)
np.save("testX", testX)
#testX.dump("testX.dat")

(929615, 23)
<class 'pandas.core.frame.DataFrame'>
(931453, 40)
(929615, 40)
the test ids are:  929615
(929615, 39)


In [12]:
print(test_ids.shape)
print(test_ids.head())

(929615,)
0      15889
1    1170544
2    1170545
3    1170547
4    1170548
Name: ncodpers, dtype: int64


In [37]:
import xgboost as xgb

params = {'seed': 0,
          'colsample_bytree': 0.7,
          'silent': 1,
          'subsample': 0.5,
          'learning_rate': 0.1,
          'objective': 'binary:logistic',
          'max_depth': 10,
          'min_child_weight': 100,
          'booster': 'gbtree', 
          'eval_metric': 'mlogloss',
         }

#params = {'seed': 125,
#          'colsample_bytree': 0.7,
#          'silent': 1,
#          'subsample': 0.7,
#          'eta': 0.05,
#          'objective': 'multi:softprob',
#          'max_depth': 8,
#          'min_child_weight': 1,
#          'eval_metric': 'mlogloss',
#          'num_class' : 22
#         }

num_rounds = 50

trainX = xgb.DMatrix(trainX, label=trainY)
testX = xgb.DMatrix(testX)


In [16]:
import pandas as pd
import numpy as np

target_cols = ['ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1',
               'ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1',
               'ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1',
               'ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1',
               'ind_nom_pens_ult1','ind_recibo_ult1']

preds = np.load("preds500.npy")
preds[preds < 0.02] = 0
print(preds[6:7,:])

#combined = zip(test_ids, preds)
target_cols = np.array(target_cols)
    
final_preds = [" ".join(list(target_cols[np.nonzero(pred)])) for pred in preds]    
test_ids = np.array(np.load("test_ids.npy"))

final_df = pd.DataFrame({'ncodpers':test_ids, 'added_products':final_preds})
final_df.to_csv("final_df.csv", index=False)

[[ 0.53979647  0.          0.05524501  0.          0.          0.          0.
   0.          0.          0.08326565  0.02992202  0.          0.          0.
   0.          0.          0.08116972  0.          0.          0.03603326
   0.04820196  0.10860284]]
